In [1]:
import os
import sys
from importlib import reload
from pathlib import Path

import numpy as np
import pandas as pd
from torch.nn import ReLU
from torch_geometric.nn import GCNConv

In [2]:
# NOTE: Boilerplate setup for Jupyter imports
sys.path.append(Path(os.getcwd()).parent.as_posix())

import configs as config_module
import enhancer as enhancer_module
import encoders as encoders_module
import scheme.network as network_module
import scheme.data as data_module

reload(config_module)
reload(enhancer_module)
reload(encoders_module)
reload(network_module)
reload(data_module)

PathConfig, TrainConfig = config_module.PathConfig, config_module.TrainConfig
Enhancer = enhancer_module.Enhancer
get_default_encoders = encoders_module.get_default_encoders
GNNConfig, EnhancerData = network_module.GNNConfig, data_module.EnhancerData

In [3]:
path_config = PathConfig(
    data_dir="../../data"
)

with open(path_config.target_data, "rb") as f:
    unpacked = np.load(f)

    # NOTE: Target dimensions
    data = EnhancerData(
        unpacked["data"],
        unpacked["target"].reshape(-1),
        unpacked["spatial"],
    )

In [4]:
gnn_config = GNNConfig(
    activation=ReLU,
    activation_args={},

    conv_operator=GCNConv,
    conv_args={ },

    encoder_scheme=[data.features.shape[1], 256, 256],
    estimator_scheme=[128, 128, np.unique(data.target).shape[0]],
)

In [5]:
encoders = [
    encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist",
    ),
    encoders_module.ReprEncoder(
        neighbor_rate=0.7,
        cache_dir=path_config.edge_cache,
        note="cora_repr",
    ),
]

result = Enhancer.compare_builders(data, gnn_config, encoders)

cora_dist test: Loss = 1.5415e+00 | Accuracy = 0.6715867158671587
cora_repr test: Loss = 1.6239e+00 | Accuracy = 0.4280442804428044
DistEncoder test: Loss = 1.3142e+00 | Accuracy = 0.6715867158671587
ReprEncoder test: Loss = 1.6194e+00 | Accuracy = 0.4280442804428044


In [6]:
print(result)

Option         accuracy_score    f1_score
-----------  ----------------  ----------
cora_dist            0.671587    0.643595
cora_repr            0.428044    0.333933
DistEncoder          0.671587    0.644066
ReprEncoder          0.428044    0.321041
